In [3]:
using Oscar

In [4]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [5]:
function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [6]:
function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end
    

convertIncidenceMatrix (generic function with 1 method)

In [7]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 

coneListFormat (generic function with 1 method)

In [60]:
function findBarycenter(s,X)
    rays = rowMinors(Array(X.RAYS), s)
    dim=size(rays,2)
    bary=zeros(Polymake.Rational,dim,1)
    for i in 1:size(rays,1)
        bary+=rays[i,:]
    end
    bary=Polymake.common.primitive(bary)
    return bary
end

function toric_blowup(s, X, v)
    s = [i + 1 for i in s]
    if v==nothing
        v=findBarycenter(s,X)
    end
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = [coneList[i] for i in starIndex]
    rayMatrix = X.RAYS
    
    lattice = X.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
        rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
        for cone in rank_c_subcones
            new_cone = [i+1 for i in cone]
            if all((i -> i in t).(new_cone))
                push!(clStar, new_cone)
            end
        end
    end
    clStar = unique(clStar)
    
    n = size(rayMatrix, 1) + 1
    coneList = filter(x -> !(x in star), coneList)
    
    if length(s) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                push!(newCones, sort(push!(t, s[1])))
            end
        end
        finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
        return Polymake.fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            push!(newCones, push!(t, n))
        end
    end
    finalRays = vcat((X.RAYS),transpose(v))
    finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
    
    return Polymake.fulton.NormalToricVariety(INPUT_RAYS = finalRays, INPUT_CONES = finalCones)
end

toric_blowup (generic function with 1 method)

In [23]:
M=[1 0;0 1]
M[1,:]

2-element Array{Int64,1}:
 0
 1

In [40]:
tv = Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0; 0 1]; INPUT_CONES=[[0,1]])

type: NormalToricVariety

INPUT_CONES
	{0 1}

INPUT_RAYS
	1 0
	0 1

In [42]:
tv.SMOOTH

true

In [61]:
findBarycenter([1,2],tv)

pm::Matrix<pm::Integer>
1
1


In [62]:
Z=toric_blowup([0,1], tv, nothing)

type: NormalToricVariety

INPUT_CONES
	{0 2}
	{1 2}

INPUT_RAYS
	1 0
	0 1
	1 1

In [20]:
out.SMOOTH

true

In [ ]:


coneList = convertIncidenceMatrix(tv.MAXIMAL_CONES)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
coneList = convert.(Array{Int64, 1}, coneList)
starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
star = rowMinors(coneList, starIndex)
println(star)
rayMatrix = tv.RAYS

In [14]:
append!([0], [1 2 3])

4-element Array{Int64,1}:
 0
 1
 2
 3

In [16]:
hi = []
append!(hi, [1,2,3])
print(hi)

Any[1, 2, 3]